# **Project Deacription**

This notebook presents a Deep Reinforcement Learning (DRL) solution for the continuous problem of dynamic portfolio optimization.

The goal is to train an autonomous agent to manage and rebalance a portfolio of assets over time, aiming to maximize risk-adjusted returns in a complex, non-stationary market environment.

 **Methodology**:

 **Proximal Policy Optimization** (PPO)

 The core of this project is an agent trained using the Proximal Policy Optimization (PPO) algorithm.

 **Environment**:

 The financial market is modeled as a custom Reinforcement Learning Environment, where the agent interacts with a set of 20 European assets (from Germany and Italy) plus a cash holding.

 **Observation Space** (State):

 The agent's decision-making is informed by a comprehensive 120-feature state vector.

 This includes various technical indicators such as:

 **Momentum Metrics**: To capture price trends.
 **Relative Strength Indicators**: To measure an asset's performance against the broader market.

 **Market Volatility Metrics**: To assess market risk and uncertainty.

 **Action Space**

 At each time step, the agent outputs a vector of portfolio weights, determining the allocation of capital across the 20 assets and cash.

 **Reward Function**

 The agent is guided by a novel reward function based on the **Adjusted Sharpe Ratio**.

 This function is designed to penalize the agent not just for low returns, but also for taking on excessive risk or for underperforming the market benchmark, the **STOXX** index.

 **WARNING**:

 This notebook is not a financial advisor.



In [1]:
pip install stable-baselines3 torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 3.1 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from collections import deque
import random
import time
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from scipy.special import softmax


In [10]:



TICKERS_BY_COUNTRY = {
    'Germany': ['ALV.DE', 'DBK.DE', 'CBK.DE', 'HAG.DE', 'DB1.DE', 'FPE.DE', 'DHER.DE', 'MUV2.DE', 'VNA.DE', 'SDF.DE'],
    'Italy': ['ISP.MI', 'UCG.MI', 'BAMI.MI', 'BMED.MI', 'FBK.MI', 'G.MI', 'AZM.MI', 'PST.MI', 'RACE.MI', 'IP.MI']
}
ALL_TICKERS = [ticker for country in TICKERS_BY_COUNTRY.values() for ticker in country]
TICKERS = ALL_TICKERS
N_ASSETS = len(TICKERS)
START_DATE = '2021-01-01'
END_DATE = '2025-09-30'

MARKET_TICKER = '^STOXX'
RISK_FREE_RATE_ANNUAL = 0.015
TRADING_DAYS_PER_YEAR = 252


class ParticipantVisibleError(Exception):
    pass

def score_multi_asset(solution: pd.DataFrame, submission: pd.DataFrame) -> float:


    if not submission.index.equals(solution.index):
        solution = solution.reindex(submission.index)


    temp_df = solution.copy()
    temp_df['strategy_returns'] = submission['prediction']


    strategy_excess_returns = temp_df['strategy_returns'] - temp_df['risk_free_rate']
    strategy_std_excess = strategy_excess_returns.std()

    if strategy_std_excess == 0:
        return -100.0


    mean_excess_return_daily = strategy_excess_returns.mean()
    strategy_mean_excess_return = mean_excess_return_daily * TRADING_DAYS_PER_YEAR
    sharpe = mean_excess_return_daily / strategy_std_excess * np.sqrt(TRADING_DAYS_PER_YEAR)


    strategy_volatility = float(temp_df['strategy_returns'].std() * np.sqrt(TRADING_DAYS_PER_YEAR) * 100)


    market_excess_returns = temp_df['forward_returns_PORTFOLIO'] - temp_df['risk_free_rate']
    mean_market_excess_return_daily = market_excess_returns.mean()
    market_mean_excess_return = mean_market_excess_return_daily * TRADING_DAYS_PER_YEAR

    market_std = temp_df['forward_returns_PORTFOLIO'].std()
    market_volatility = float(market_std * np.sqrt(TRADING_DAYS_PER_YEAR) * 100)

    if market_volatility == 0:
        return -100.0


    excess_vol = max(0, strategy_volatility / market_volatility - 0.9)
    vol_penalty = 1 + excess_vol


    return_gap = max(
        0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100,
    )
    return_penalty = 1 + (return_gap**2) / 100


    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)



def prepare_multi_asset_data(tickers, market_ticker, start_date, end_date, risk_free_rate_annual):


    print(f"--- 1. Fetching and Preparing Data for {len(tickers)} Assets using {market_ticker} as Benchmark ---")


    all_symbols = tickers + [market_ticker]

    if isinstance(market_ticker, list):
        market_ticker = market_ticker[0]

    data_raw = yf.download(all_symbols, start=start_date, end=end_date, auto_adjust=False)['Close']


    market_close = data_raw[market_ticker]
    market_returns = market_close.pct_change()
    risk_free_rate = risk_free_rate_annual / TRADING_DAYS_PER_YEAR


    data_frames = []

    for ticker in tickers:
        df_asset = pd.DataFrame(index=data_raw.index)
        df_asset['close'] = data_raw[ticker]
        df_asset['market_return'] = market_returns
        df_asset['risk_free_rate'] = risk_free_rate


        df_asset['vol_adjusted_momentum'] = df_asset['close'].pct_change(20) / df_asset['close'].rolling(20).std()


        df_asset['relative_strength'] = df_asset['close'].pct_change(1) - df_asset['market_return'].shift(1)


        market_vol_20d = df_asset['market_return'].rolling(window=20).std()
        df_asset['market_vol_change'] = market_vol_20d.pct_change(20)


        df_asset['ema_10'] = df_asset['close'].ewm(span=10, adjust=False).mean()
        df_asset['ema_ratio'] = df_asset['ema_10'] / df_asset['close'].rolling(20).mean()


        df_asset['momentum_5d'] = df_asset['close'].pct_change(5)


        df_asset['forward_returns'] = df_asset['close'].pct_change().shift(-1)


        df_asset = df_asset.drop(columns=['close'])


        df_asset.columns = [f'{col}_{ticker}' for col in df_asset.columns]

        data_frames.append(df_asset)


    final_df = pd.concat(data_frames, axis=1)


    forward_return_cols = [f'forward_returns_{t}' for t in tickers]
    final_df['forward_returns_PORTFOLIO'] = final_df[forward_return_cols].mean(axis=1)


    final_df.dropna(subset=forward_return_cols + ['forward_returns_PORTFOLIO'], inplace=True)
    final_df.dropna(inplace=True)


    all_features = [col for col in final_df.columns if not col.startswith(('forward_returns', 'market_return', 'risk_free_rate'))]

    scaler = StandardScaler()
    final_df[all_features] = scaler.fit_transform(final_df[all_features])

    print(f"Data Ready. Total days: {len(final_df)}. Total features: {len(all_features)}")
    return final_df, all_features, forward_return_cols



class MultiAssetPortfolioEnv(gym.Env):

    def __init__(self, data_df, features, forward_return_cols, n_assets):
        super(MultiAssetPortfolioEnv, self).__init__()
        self.df = data_df.copy().reset_index(drop=True)
        self.features = features
        self.forward_return_cols = forward_return_cols
        self.n_assets = n_assets
        self.n_states = len(features)


        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(n_assets + 1,), dtype=np.float32)


        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_states,), dtype=np.float32)


        self.strategy_returns = deque(maxlen=TRADING_DAYS_PER_YEAR)
        self.daily_returns_list = []

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.strategy_returns.clear()
        self.daily_returns_list = []
        return self._get_state(), {}

    def _get_state(self):

        return self.df.loc[self.current_step, self.features].values.astype(np.float32)

    def step(self, raw_action: np.ndarray):


        weights = softmax(raw_action)


        asset_weights = weights[:-1]
        cash_weight = weights[-1]


        row = self.df.loc[self.current_step]
        r_f = row[f'risk_free_rate_{TICKERS[0]}']


        asset_returns = row[self.forward_return_cols].values


        daily_return = (cash_weight * r_f) + np.dot(asset_weights, asset_returns)

        self.strategy_returns.append(daily_return)
        self.daily_returns_list.append(daily_return)


        reward = 0.0
        if len(self.strategy_returns) >= 20:
            rolling_returns = np.array(self.strategy_returns)
            excess_returns = rolling_returns - r_f
            mean_excess = np.mean(excess_returns)
            std = np.std(excess_returns)

            if std == 0:
                 sharpe_proxy = -1.0
            else:
                 sharpe_proxy = mean_excess / std * np.sqrt(TRADING_DAYS_PER_YEAR)


            reward = sharpe_proxy * 100


            if np.prod(1 + rolling_returns) - 1 < 0:
                reward += (np.prod(1 + rolling_returns) - 1) * 500

        else:

            reward = daily_return * 100


        self.current_step += 1
        done = self.current_step >= len(self.df) - 1

        next_state = self._get_state() if not done else np.zeros(self.observation_space.shape, dtype=np.float32)

        return next_state, reward, done, False, {}

    def get_final_score(self):



        solution_data = self.df.iloc[:len(self.daily_returns_list)][[f'risk_free_rate_{TICKERS[0]}', 'forward_returns_PORTFOLIO']].rename(
            columns={f'risk_free_rate_{TICKERS[0]}': 'risk_free_rate'}
        )


        submission_data = pd.DataFrame(
            {'prediction': self.daily_returns_list},
            index=solution_data.index
        )


        return score_multi_asset(solution_data, submission_data)



class TradingCallback(BaseCallback):
    def __init__(self, check_freq: int, verbose: int = 1):
        super(TradingCallback, self).__init__(verbose)
        self.check_freq = check_freq

    def _on_step(self) -> bool:
        return True

def train_ppo_agent(env, total_timesteps):



    env = DummyVecEnv([lambda: env])


    model = PPO(
        "MlpPolicy",
        env,
        verbose=1,
        seed=42,

        learning_rate=0.0001,
        n_steps=2048,
        batch_size=64
    )

    print(f"\n--- 2. Training Multi-Asset DRL (PPO) Agent for {total_timesteps} timesteps ---")


    model.learn(
        total_timesteps=total_timesteps,
        callback=TradingCallback(check_freq=5000)
    )
    print("DRL Training Complete.")

    return model

def evaluate_drl_policy(model, env):

    print("\n--- 5. Evaluation (Policy Test on Validation Data) ---")

    obs, _ = env.reset()
    done = False

    for i in range(len(env.df) - 1):

        action, _states = model.predict(obs, deterministic=True)

        obs, reward, terminated, truncated, info = env.step(action)
        if terminated or truncated: break

    final_sharpe = env.get_final_score()
    print(f" Validation Complete. Total Days: {env.current_step}")
    return final_sharpe


if __name__ == "__main__":


    TOTAL_TIMESTEPS = 750000


    final_data, features, forward_return_cols = prepare_multi_asset_data(
        TICKERS, MARKET_TICKER, START_DATE, END_DATE, RISK_FREE_RATE_ANNUAL
    )


    train_size = int(len(final_data) * 0.7)
    train_data = final_data.iloc[:train_size]
    validation_data = final_data.iloc[train_size:]

    print(f"\nDataset split: Train ({len(train_data)} days), Validation ({len(validation_data)} days)")


    train_env = MultiAssetPortfolioEnv(
        data_df=train_data,
        features=features,
        forward_return_cols=forward_return_cols,
        n_assets=N_ASSETS
    )


    trained_policy = train_ppo_agent(train_env, total_timesteps=TOTAL_TIMESTEPS)


    validation_env = MultiAssetPortfolioEnv(
        data_df=validation_data,
        features=features,
        forward_return_cols=forward_return_cols,
        n_assets=N_ASSETS
    )

    final_validation_sharpe = evaluate_drl_policy(trained_policy, validation_env)

    print(f"\n--- DRL Strategy Pipeline Result (PPO, {TOTAL_TIMESTEPS} steps) ✨ ---")
    print(f"Number of Assets: {N_ASSETS} (Stocks + Cash)")
    print(f"Market Benchmark: {MARKET_TICKER}")
    print(f"Total Features in State Space: {len(features)}")
    print(f"Validation Period Adjusted Sharpe: **{final_validation_sharpe:.4f}**")

[                       0%                       ]

--- 1. Fetching and Preparing Data for 20 Assets using ^STOXX as Benchmark ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
[*****                 10%                       ]  2 of 21 completed/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return dateti

Data Ready. Total days: 1071. Total features: 120

Dataset split: Train (749 days), Validation (322 days)
Using cpu device

--- 2. Training Multi-Asset DRL (PPO) Agent for 750000 timesteps ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


DRL Training Complete.

--- 5. Evaluation (Policy Test on Validation Data) ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


 Validation Complete. Total Days: 321

--- DRL Strategy Pipeline Result (PPO, 750000 steps) ✨ ---
Number of Assets: 20 (Stocks + Cash)
Market Benchmark: ^STOXX
Total Features in State Space: 120
Validation Period Adjusted Sharpe: **1.3645**


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


**Results**

The DRL agent was trained for 750,000 steps and validated on an unseen (out-of-sample) data set.

**Key Result**:

The strategy achieved a Validation Period **Adjusted Sharpe of 1.3645**.

This metric indicates that the PPO agent successfully learned a policy that generates competitive returns while maintaining rigorous risk control, outperforming the benchmark on a risk-adjusted basis.

**WARNING**:

This notebook is not a financial advisor.